In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import matplotlib.pyplot as plt
import torchvision as T
import torch
from sklearn import preprocessing
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from torch.utils.data import Dataset, DataLoader
import math
import itertools
import os
import sys
root_path = os.path.abspath(os.path.join('../..')) # <- adjust such that root_path always points at the root project dir (i.e. if current file is two folders deep, use '../..'). 
if root_path not in sys.path:
    sys.path.append(root_path)
import database_server.db_utilities as dbu 
import pickle as pkl
from sklearn.decomposition import PCA
from Help_functions import preprocess, game_dict, inputs, inputs_2seas, club_dict, points_and_co, points_and_co_oppon, data_to_lstm, Sport_pred_2LSTM_1, predict
import random

In [2]:
torch.manual_seed(69420)
random.seed(69420)
np.random.seed(69420)

In [3]:
query_str = """
SELECT ms.*, 
       m.schedule_date, m.schedule_time, m.schedule_round, m.schedule_day,
       w.annual_wages_eur AS annual_wage_team, 
       w.weekly_wages_eur AS weekly_wages_eur,
       w.annual_wages_eur/w.n_players AS annual_wage_player_avg
FROM matchstats ms 
LEFT JOIN matches m ON ms.match_id = m.id
LEFT JOIN teamwages w ON ms.team_id = w.team_id
AND     ms.season_str = w.season_str
ORDER BY m.schedule_date DESC, m.schedule_time DESC; 
"""

df_allinfo = dbu.select_query(query_str)

In [4]:
new_data_test = preprocess(df_allinfo)

In [5]:
scale_df = new_data_test.data_frame.copy()
#scale_df["stad_capac"] = 0
for team in scale_df.team_id.unique():
    max_attend = max(scale_df[(scale_df.team_id == team) & (scale_df.venue == new_data_test.return_dicts("venue")["Home"])].attendance)
    scale_df.loc[(scale_df.team_id == team) & (scale_df.venue == new_data_test.return_dicts("venue")["Home"]),"stad_capac"] = scale_df[(scale_df.team_id == team) & (scale_df.venue == new_data_test.return_dicts("venue")["Home"])].attendance.apply(lambda x: x/max_attend)
    scale_df.loc[(scale_df.opponent_id == team) & (scale_df.venue == new_data_test.return_dicts("venue")["Away"]),"stad_capac"] = scale_df[(scale_df.opponent_id == team) & (scale_df.venue == new_data_test.return_dicts("venue")["Away"])].attendance.apply(lambda x: x/max_attend)

In [6]:
scale_df

,venue,result,gf,ga,xg,xga,attendance,captain,formation,referee,...,misc_aerialduels_lost,misc_aerialduels_won_perc,schedule_date,schedule_time,schedule_round,schedule_day,annual_wage_team,weekly_wages_eur,annual_wage_player_avg,stad_capac
0,0,2,1,1,1.6,2.5,16585.0,370,2,1,...,12,52.0,2023-06-04,1,38,0,12225000.0,235096.0,489000.0,0.548518
1,1,2,1,1,1.2,0.6,54087.0,923,3,2,...,4,71.4,2023-06-04,1,38,0,42500000.0,817308.0,1770833.0,0.972998
2,0,0,0,1,0.8,1.3,23266.0,406,1,3,...,9,57.1,2023-06-04,1,38,0,9210000.0,177115.0,341111.0,0.930640
3,1,2,1,1,2.5,1.6,16585.0,679,4,1,...,13,48.0,2023-06-04,1,38,0,12015000.0,231058.0,445000.0,0.548518
4,0,2,3,3,0.6,1.8,14213.0,590,3,4,...,22,29.0,2023-06-04,1,38,0,15790000.0,303654.0,717727.0,0.423485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21701,1,0,0,4,0.5,1.8,1979.0,1053,2,78,...,15,42.3,2017-08-05,11,1,1,10804000.0,207769.0,348516.0,0.033986
21702,0,1,2,0,2.6,0.3,46898.0,980,5,140,...,5,54.5,2017-08-05,28,1,1,202564000.0,3895462.0,5787542.0,0.978550
21703,1,0,0,2,0.3,2.6,46898.0,575,9,140,...,6,45.5,2017-08-05,28,1,1,12330000.0,237115.0,362647.0,0.978550
21704,0,1,3,2,2.0,0.3,13572.0,299,4,28,...,20,58.3,2017-08-04,0,1,2,68350000.0,1314423.0,1798684.0,0.815527


In [7]:
#scale_df = new_data_test.data_frame.copy()

liste = ['schedule_time', 'schedule_round', 'schedule_day', 'result', 'gf', 'ga', 'xg', 'xga', 'formation', 
         'referee', 'season_str', 'league_id', 'team_id', 'opponent_id', 'match_id', 'id', 'fbref_id', 
         'home_team_id', 'away_team_id', 'schedule_date', 'venue', 'captain',]

cols_to_scale = list(set(list(scale_df.columns)).difference(liste))
object_ = StandardScaler()

scale_df.loc[:,cols_to_scale] = object_.fit_transform(scale_df.loc[:,cols_to_scale])
scale_df.head()

,venue,result,gf,ga,xg,xga,attendance,captain,formation,referee,...,misc_aerialduels_lost,misc_aerialduels_won_perc,schedule_date,schedule_time,schedule_round,schedule_day,annual_wage_team,weekly_wages_eur,annual_wage_player_avg,stad_capac
0,0,2,1,1,1.6,2.5,-0.712704,370,2,1,...,-0.558475,0.186329,2023-06-04,1,38,0,-0.748507,-0.748507,-0.411856,-0.962527
1,1,2,1,1,1.2,0.6,1.303480,923,3,2,...,-1.695420,1.995457,2023-06-04,1,38,0,-0.281729,-0.281729,0.444207,0.909555
2,0,0,0,1,0.8,1.3,-0.353520,406,1,3,...,-0.984829,0.661925,2023-06-04,1,38,0,-0.794992,-0.794992,-0.510622,0.722744
3,1,2,1,1,2.5,1.6,-0.712704,679,4,1,...,-0.416357,-0.186687,2023-06-04,1,38,0,-0.751744,-0.751744,-0.441241,-0.962527
4,0,2,3,3,0.6,1.8,-0.840228,590,3,4,...,0.862707,-1.958514,2023-06-04,1,38,0,-0.693542,-0.693542,-0.259102,-1.513961


In [8]:
ohe_team = OneHotEncoder()
to_ohe_team = scale_df.loc[:, ["team_id", "opponent_id"]]
ohe_team.fit(to_ohe_team)

codes = ohe_team.transform(to_ohe_team).toarray()
feature_names = ohe_team.get_feature_names(['team_id', 'opponent_id'])

scale_df = pd.concat([scale_df, 
               pd.DataFrame(codes, columns = feature_names).astype(int)], axis=1)
##########################################
ohe_ligue = OneHotEncoder()
to_ohe_ligue = scale_df.loc[:,["league_id"]]
ohe_ligue.fit(to_ohe_ligue)

codes = ohe_ligue.transform(to_ohe_ligue).toarray()
feature_names = ohe_ligue.get_feature_names(['league_id'])

scale_df = pd.concat([scale_df, 
               pd.DataFrame(codes, columns = feature_names).astype(int)], axis=1)


In [9]:
clubs = club_dict(scale_df)

In [10]:
result_dict = new_data_test.return_dicts("result")
clubs = points_and_co(clubs, result_dict)

In [11]:
clubs = points_and_co_oppon(clubs, result_dict)

In [12]:
game_train, game_valid = game_dict(scale_df)

In [13]:
abcdefg = list(scale_df.columns)
abc = abcdefg[:abcdefg.index("annual_wage_player_avg")+1]
defg = abcdefg[abcdefg.index("annual_wage_player_avg")+1:]

rearange_list = ['result', 'gf', 'ga', 'goal_diff', 'xg', 'xga', 'shooting_standard_gls',
       'shooting_standard_sh', 'shooting_standard_sot',
       'shooting_standard_sot_perc', 'shooting_standard_g_per_sh',
       'shooting_standard_g_per_sot', 'shooting_standard_dist',
       'shooting_standard_fk', 'shooting_standard_pk',
       'shooting_standard_pkatt', 'shooting_expected_npxg',
       'shooting_expected_npxg_per_sh', 'shooting_expected_g_minus_xg',
       'shooting_expected_npg_minus_xg', 'keeper_performance_sota',
       'keeper_performance_saves', 'keeper_performance_save_perc',
       'keeper_performance_cs', 'keeper_performance_psxg',
       'keeper_performance_psxg_plus_minus', 'keeper_penaltykicks_pkatt',
       'keeper_penaltykicks_pka', 'keeper_penaltykicks_pksv',
       'keeper_penaltykicks_pkm', 'keeper_launched_cmp', 'keeper_launched_att',
       'keeper_launched_cmp_perc', 'keeper_passes_att', 'keeper_passes_thr',
       'keeper_passes_launch_perc', 'keeper_passes_avglen',
       'keeper_goalkicks_att', 'keeper_goalkicks_launch_perc',
       'keeper_goalkicks_avglen', 'keeper_crosses_opp', 'keeper_crosses_stp',
       'keeper_crosses_stp_perc', 'keeper_sweeper_number_opa',
       'keeper_sweeper_avgdist', 'passing_total_cmp', 'passing_total_att',
       'passing_total_cmp_perc', 'passing_total_totdist',
       'passing_total_prgdist', 'passing_short_cmp', 'passing_short_att',
       'passing_short_cmp_perc', 'passing_medium_cmp', 'passing_medium_att',
       'passing_medium_cmp_perc', 'passing_long_cmp', 'passing_long_att',
       'passing_long_cmp_perc', 'passing_attacking_ast',
       'passing_attacking_xag', 'passing_attacking_xa', 'passing_attacking_kp',
       'passing_attacking_1_per_3', 'passing_attacking_ppa',
       'passing_attacking_crspa', 'passing_attacking_prgp',
       'passing_types_passtypes_live', 'passing_types_passtypes_dead',
       'passing_types_passtypes_fk', 'passing_types_passtypes_tb',
       'misc_aerialduels_won_perc', 'stad_capac', 'attendance', 'points', 'mean_points',
       'weekly_wages_eur', 'season_str',  'league_id', 'venue', 'team_id',
       'opponent_id', 'last_results', 'oppon_points', 'oppon_mean_points', 'oppon_wages', 'schedule_round',
        'captain', 'formation', 'referee',  'match_id', 'schedule_date', 'schedule_time',
        'schedule_day', 'annual_wage_team', 'annual_wage_player_avg',]

rearange_list = list(itertools.chain.from_iterable(defg if item == "team_id" else [item] for item in rearange_list))

del rearange_list[rearange_list.index("opponent_id")]

In [14]:
def inputs_2seas(games, clubs, rearrange_list, scale_df):
    """
    Creates input tensors and targets to feed to the LSTM
    Inputs: games, dictionary containing every game
            clubs, containing a dataframe for every club with all its games
            rearrange_list, a list of column names to reorder based on ones needs
            scale_df, the dataframe containing all games
    Output: list of lists  (1. all data to games club X played in current season till day Z 
                            2. all data to games club Y played in current season till day Z 
                            3. all results for club X in current season (result, goals scored, goals conceded, goal difference
                            4. all results for club Y in current season (result, goals scored, goals conceded, goal difference
    """
    
    # [[input1], [input2], [result1], [gf1], [ga1], [g_diff1], [result2], [gf2], [ga2], [g_diff2]]
    lstm_inputs = [[], [], [], [], []] 
    
    
    for i in range(7,len(games)):  # start of prediction in game 7
        print(i)
        
        for game in games[i]:
            
            if game in scale_df[scale_df.season_str == 0].match_id.unique():
                pass
            
            else:
                season = scale_df[scale_df.match_id == game].season_str.iloc[0]
                ## Team 1
                # get ids of participating teams in game 
                team1 = scale_df[scale_df.match_id == game].iloc[0].team_id
                team2 = scale_df[scale_df.match_id == game].iloc[1].team_id
                
                if (team1 in scale_df[scale_df.season_str == season - 1].team_id.unique()) and (team2 in scale_df[scale_df.season_str == season - 1].team_id.unique()):
                    # get dataframe containing all games of club and reorder according to our need
                    df_team1 = clubs[team1]
                    df_team1 = df_team1[rearrange_list]

                    # get season in which game took place
                      

                    # create dataframe containing all games of season prior to the game we want to predict
                    df_team1_past = df_team1.loc[ \
                        df_team1[df_team1.season_str == season - 1].iloc[0].name: # index of first game in seasons
                        df_team1[df_team1.match_id == game].index.values[0],  # index of game we want to predict
                        :]  # all columns

                    ## Team 2
                    # get dataframe containing all games of club and reorder according to our need
                    df_team2 = clubs[team2]
                    df_team2 = df_team2[rearrange_list]
                    df_team2_past = df_team2.loc[
                        df_team2[df_team2.season_str == season - 1].iloc[0].name:  # index of first game in seasons
                        df_team2[df_team2.match_id == game].index.values[0],  # index of game we want to predict
                        :]  # all columns

                    # create np array with zero to store data
                    np_team1 = np.zeros([len(games)*2, df_team1_past.loc[:,"xg":"schedule_round"].shape[1]])
                    np_team2 = np.zeros([len(games)*2, df_team2_past.loc[:,"xg":"schedule_round"].shape[1]])


                    # insert data into array (back to front) such that all input into the lstm has the same length (padding)
                    np_team1[- len(df_team1_past):, :-df_team1.loc[:, "weekly_wages_eur":"schedule_round"].shape[1]] = df_team1_past.loc[:,"xg": "mean_points"]
                    np_team2[- len(df_team2_past):, :-df_team1.loc[:, "weekly_wages_eur":"schedule_round"].shape[1]] = df_team2_past.loc[:,"xg": "mean_points"]


                    # if last game of the season, no do not add as no future results to predict
                    if (df_team1.iloc[-1].name == df_team1_past.iloc[-1].name) or (df_team2.iloc[-1].name == df_team2_past.iloc[-1].name):
                        pass

                    # if not last game, add data from last game to input array
                    else:
                        # 
                        np_team1[- len(df_team1_past):, -df_team1.loc[:, "weekly_wages_eur":"schedule_round"].shape[1]:] = df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1].name,"weekly_wages_eur":"schedule_round"]  
                        np_team2[- len(df_team2_past):, -df_team1.loc[:, "weekly_wages_eur":"schedule_round"].shape[1]:] = df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1].name,"weekly_wages_eur":"schedule_round"]  

                        results1 = np.zeros((len(games) * 2, 4))
                        results2 = np.zeros((len(games) * 2, 4))
                        #print(result1)
                        res1 = len(df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1].name,"result"])
                        res2 = len(df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1].name,"result"])
                        results1[-res1:, 0] = df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1].name,"result"]
                        results1[-res1:, 1] = df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1].name,"gf"]
                        results1[-res1:, 2] = df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1].name,"ga"]
                        results1[-res1:, 3] = df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1].name,"goal_diff"]
                        results2[-res2:, 0] = df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1].name,"result"]
                        results2[-res2:, 1] = df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1].name,"gf"]
                        results2[-res2:, 2] = df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1].name,"ga"]
                        results2[-res2:, 3] = df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1].name,"goal_diff"]
                        #fut_feature1 = np.array(df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1,2:].values)
                        #fut_feature2 = np.array(df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1,2:].values)
                        #print(result1)
                        result1 = torch.nn.functional.one_hot(torch.tensor(results1[:,0]).long(), num_classes = 3)
                        result2 = torch.nn.functional.one_hot(torch.tensor(results2[:,0]).long(), num_classes = 3)
                        lstm_inputs[0].append(torch.tensor(np_team1))
                        lstm_inputs[1].append(torch.tensor(np_team2))
                        lstm_inputs[2].append(result1)
                        lstm_inputs[3].append(result2)
                        #lstm_inputs[4].append(fut_feature1)
                        #lstm_inputs[5].append(fut_feature2)
                        
                elif (team1 in scale_df[scale_df.season_str == season - 1].team_id.unique()) and (team2 not in scale_df[scale_df.season_str == season - 1].team_id.unique()):
                     # get dataframe containing all games of club and reorder according to our need
                    df_team1 = clubs[team1]
                    df_team1 = df_team1[rearrange_list]

                    # get season in which game took place
                    season = df_team1[df_team1.match_id == game].season_str.values[0]  

                    # create dataframe containing all games of season prior to the game we want to predict
                    df_team1_past = df_team1.loc[ \
                        df_team1[df_team1.season_str == season - 1].iloc[0].name: # index of first game in seasons
                        df_team1[df_team1.match_id == game].index.values[0],  # index of game we want to predict
                        :]  # all columns

                    ## Team 2
                    # get dataframe containing all games of club and reorder according to our need
                    df_team2 = clubs[team2]
                    df_team2 = df_team2[rearrange_list]

                    df_team2_past = df_team2.loc[
                        df_team2[df_team2.season_str == season].iloc[0].name:  # index of first game in seasons
                        df_team2[df_team2.match_id == game].index.values[0],  # index of game we want to predict
                        :]  # all columns

                    # create np array with zero to store data
                    np_team1 = np.zeros([len(games)*2, df_team1_past.loc[:,"xg":"schedule_round"].shape[1]])
                    np_team2 = np.zeros([len(games)*2, df_team2_past.loc[:,"xg":"schedule_round"].shape[1]])


                    # insert data into array (back to front) such that all input into the lstm has the same length (padding)
                    np_team1[- len(df_team1_past):, :-df_team1.loc[:, "stad_capac":"schedule_round"].shape[1]] = df_team1_past.loc[:,"xg": "mean_points"]
                    np_team2[- len(df_team2_past):, :-df_team1.loc[:, "stad_capac":"schedule_round"].shape[1]] = df_team2_past.loc[:,"xg": "mean_points"]


                    # if last game of the season, no do not add as no future results to predict
                    if (df_team1.iloc[-1].name == df_team1_past.iloc[-1].name) or (df_team2.iloc[-1].name == df_team2_past.iloc[-1].name):
                        pass


                    # if not last game, add data from last game to input array
                    else:
                        # 
                        np_team1[- len(df_team1_past):, -df_team1.loc[:, "stad_capac":"schedule_round"].shape[1]:] = df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1].name,"stad_capac":"schedule_round"]  
                        np_team2[- len(df_team2_past):, -df_team1.loc[:, "stad_capac":"schedule_round"].shape[1]:] = df_team2.loc[df_team2[df_team2.season_str == season].iloc[1].name:df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1].name,"stad_capac":"schedule_round"]  

                        results1 = np.zeros((len(games) * 2, 4))
                        results2 = np.zeros((len(games) * 2, 4))
                        #print(result1)
                        res1 = len(df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1].name,"result"])
                        res2 = len(df_team2.loc[df_team2[df_team2.season_str == season].iloc[1].name:df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1].name,"result"])
                        results1[-res1:, 0] = df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1].name,"result"]
                        results1[-res1:, 1] = df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1].name,"gf"]
                        results1[-res1:, 2] = df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1].name,"ga"]
                        results1[-res1:, 3] = df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1].name,"goal_diff"]
                        results2[-res2:, 0] = df_team2.loc[df_team2[df_team2.season_str == season].iloc[1].name:df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1].name,"result"]
                        results2[-res2:, 1] = df_team2.loc[df_team2[df_team2.season_str == season].iloc[1].name:df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1].name,"gf"]
                        results2[-res2:, 2] = df_team2.loc[df_team2[df_team2.season_str == season].iloc[1].name:df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1].name,"ga"]
                        results2[-res2:, 3] = df_team2.loc[df_team2[df_team2.season_str == season].iloc[1].name:df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1].name,"goal_diff"]
                        #fut_feature1 = np.array(df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1,2:].values)
                        #fut_feature2 = np.array(df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1,2:].values)
                        #print(result1)
                        result1 = torch.nn.functional.one_hot(torch.tensor(results1[:,0]).long(), num_classes = 3)
                        result2 = torch.nn.functional.one_hot(torch.tensor(results2[:,0]).long(), num_classes = 3)
                        lstm_inputs[0].append(torch.tensor(np_team1))
                        lstm_inputs[1].append(torch.tensor(np_team2))
                        lstm_inputs[2].append(result1)
                        lstm_inputs[3].append(result2)
                        #lstm_inputs[4].append(fut_feature1)
                        #lstm_inputs[5].append(fut_feature2)
                        
                elif (team1 not in scale_df[scale_df.season_str == season - 1].team_id.unique()) and (team2 in scale_df[scale_df.season_str == season - 1].team_id.unique()):
                     # get dataframe containing all games of club and reorder according to our need
                    df_team1 = clubs[team1]
                    df_team1 = df_team1[rearrange_list]

                    # get season in which game took place
                    season = df_team1[df_team1.match_id == game].season_str.values[0]  

                    # create dataframe containing all games of season prior to the game we want to predict
                    df_team1_past = df_team1.loc[ \
                        df_team1[df_team1.season_str == season].iloc[0].name: # index of first game in seasons
                        df_team1[df_team1.match_id == game].index.values[0],  # index of game we want to predict
                        :]  # all columns

                    ## Team 2
                    # get dataframe containing all games of club and reorder according to our need
                    df_team2 = clubs[team2]
                    df_team2 = df_team2[rearrange_list]

                    df_team2_past = df_team2.loc[
                        df_team2[df_team2.season_str == season - 1].iloc[0].name:  # index of first game in seasons
                        df_team2[df_team2.match_id == game].index.values[0],  # index of game we want to predict
                        :]  # all columns

                    # create np array with zero to store data
                    np_team1 = np.zeros([len(games)*2, df_team1_past.loc[:,"xg":"schedule_round"].shape[1]])
                    np_team2 = np.zeros([len(games)*2, df_team2_past.loc[:,"xg":"schedule_round"].shape[1]])


                    # insert data into array (back to front) such that all input into the lstm has the same length (padding)
                    np_team1[- len(df_team1_past):, :-df_team1.loc[:, "stad_capac":"schedule_round"].shape[1]] = df_team1_past.loc[:,"xg": "mean_points"]
                    np_team2[- len(df_team2_past):, :-df_team1.loc[:, "stad_capac":"schedule_round"].shape[1]] = df_team2_past.loc[:,"xg": "mean_points"]


                    # if last game of the season, no do not add as no future results to predict
                    if (df_team1.iloc[-1].name == df_team1_past.iloc[-1].name) or (df_team2.iloc[-1].name == df_team2_past.iloc[-1].name):
                        pass

                    # if not last game, add data from last game to input array
                    else:
                        # 
                        np_team1[- len(df_team1_past):, -df_team1.loc[:, "stad_capac":"schedule_round"].shape[1]:] = df_team1.loc[df_team1[df_team1.season_str == season].iloc[1].name:df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1].name,"stad_capac":"schedule_round"]  
                        np_team2[- len(df_team2_past):, -df_team1.loc[:, "stad_capac":"schedule_round"].shape[1]:] = df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1].name,"stad_capac":"schedule_round"]  

                        results1 = np.zeros((len(games) * 2, 4))
                        results2 = np.zeros((len(games) * 2, 4))
                        #print(result1)
                        res1 = len(df_team1.loc[df_team1[df_team1.season_str == season].iloc[1].name:df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1].name,"result"])
                        res2 = len(df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1].name,"result"])
                        results1[-res1:, 0] = df_team1.loc[df_team1[df_team1.season_str == season].iloc[1].name:df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1].name,"result"]
                        results1[-res1:, 1] = df_team1.loc[df_team1[df_team1.season_str == season].iloc[1].name:df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1].name,"gf"]
                        results1[-res1:, 2] = df_team1.loc[df_team1[df_team1.season_str == season].iloc[1].name:df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1].name,"ga"]
                        results1[-res1:, 3] = df_team1.loc[df_team1[df_team1.season_str == season].iloc[1].name:df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1].name,"goal_diff"]
                        results2[-res2:, 0] = df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1].name,"result"]
                        results2[-res2:, 1] = df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1].name,"gf"]
                        results2[-res2:, 2] = df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1].name,"ga"]
                        results2[-res2:, 3] = df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1].name,"goal_diff"]
                        #fut_feature1 = np.array(df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1,2:].values)
                        #fut_feature2 = np.array(df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1,2:].values)
                        #print(result1)
                        result1 = torch.nn.functional.one_hot(torch.tensor(results1[:,0]).long(), num_classes = 3)
                        result2 = torch.nn.functional.one_hot(torch.tensor(results2[:,0]).long(), num_classes = 3)
                        lstm_inputs[0].append(torch.tensor(np_team1))
                        lstm_inputs[1].append(torch.tensor(np_team2))
                        lstm_inputs[2].append(result1)
                        lstm_inputs[3].append(result2)
                        #lstm_inputs[4].append(fut_feature1)
                        #lstm_inputs[5].append(fut_feature2)
                        
                else:
                    pass
    return lstm_inputs

In [15]:
try print(5/0) except print(3)

SyntaxError: invalid syntax (3800355812.py, line 1)

In [ ]:
clubs[33]

In [ ]:
def two_team_inputs(team1, team2, rearrange_list, scale_df, venue_dict, attendance):
    
    lstm_inputs = [[], [], [], [], []] 
    season = int(max(scale_df.season_str))
    
    if team1 not in scale_df[scale_df.season_str == season].team_id.unique():
        #print("Team 1 did not play in the last season")
        raise KeyError("Incorrect input: Team 1 did not play in the last season")
    if team2 not in scale_df[scale_df.season_str == season].team_id.unique():
        #print("Team 2 did not play in the last season")
        raise KeyError("Incorrect input: Team 2 did not play in the last season")
                ## Team 1
                # get ids of participating teams in game 
    team1 = team1
    team2 = team2
    games = scale_df.schedule_round.unique()
    'last_results', 'oppon_points', 'oppon_mean_points'
    df_team1 = clubs[team1].copy()
    df_team1 = df_team1[rearrange_list]
    df_team1.venue.iloc[-1] = venue_dict['Home']
    df_team1.attendance = attendance
    df_team1.opponent_id.iloc[-1] = df_team2.team_id.iloc[-1]
    try:
        df_team1.last_results.iloc[-1] = df_team1[df_team1.opponent_id == team2].last_results.iloc[-1]
    except:
        df_team1.last_results.iloc[-1] = 0
    df_team1.oppon_points.iloc[-1] = df_team2.points.iloc[-1]
    df_team1.oppon_mean_points.iloc[-1] = df_team2.mean_points.iloc[-1]
    df_team1.oppon_wages.iloc[-1] = df_team2.weekly_wages_eur.iloc[-1]
    df_team1.stad_capac.iloc[-1] = attendance/max(df_team1.attendance)
    
    df_team2 = clubs[team2].copy()
    df_team2 = df_team2[rearrange_list]
    df_team2.venue.iloc[-1] = venue_dict['Away']
    df_team2.attendance = attendance
    df_team2.opponent_id.iloc[-1] = df_team1.team_id.iloc[-1]
    try:
        df_team2.last_results.iloc[-1] = df_team2[df_team2.opponent_id == team1].last_results.iloc[-1]
    except:
        df_team2.last_results.iloc[-1] = 0
    df_team2.oppon_points.iloc[-1] = df_team1.points.iloc[-1]
    df_team2.oppon_mean_points.iloc[-1] = df_team1.mean_points.iloc[-1]
    df_team2.oppon_wages.iloc[-1] = df_team1.weekly_wages_eur.iloc[-1]
    df_team2.stad_capac.iloc[-1] = attendance/max(df_team1.attendance)
    
    if (team1 in scale_df[scale_df.season_str == season - 1].team_id.unique()) and (team2 in scale_df[scale_df.season_str == season - 1].team_id.unique()):
        print("1")
        # get dataframe containing all games of club and reorder according to our need
        #df_team1 = clubs[team1]
        #df_team1 = df_team1[rearrange_list]
        # get season in which game took place


        # create dataframe containing all games of season prior to the game we want to predict
        df_team1_past = df_team1.loc[ \
            df_team1[df_team1.season_str == season - 1].iloc[0].name: # index of first game in seasons
            df_team1.iloc[-2].name,  # index of game we want to predict
            :]  # all columns
        
        ## Team 2
        # get dataframe containing all games of club and reorder according to our need
        df_team2 = clubs[team2]
        df_team2 = df_team2[rearrange_list]
        #print("team2", df_team2.shape)
        df_team2_past = df_team2.loc[
            df_team2[df_team2.season_str == season - 1].iloc[0].name:  # index of first game in seasons
            df_team2.iloc[-2].name,  # index of game we want to predict
            :]  # all columns
        
        # create np array with zero to store data
        np_team1 = np.zeros([len(games)*2, df_team1_past.loc[:,"xg":"schedule_round"].shape[1]])
        np_team2 = np.zeros([len(games)*2, df_team2_past.loc[:,"xg":"schedule_round"].shape[1]])
        #print("np_team1",np_team1.shape)

        # insert data into array (back to front) such that all input into the lstm has the same length (padding)
        np_team1[- len(df_team1_past):, :-df_team1.loc[:, "weekly_wages_eur":"schedule_round"].shape[1]] = df_team1_past.loc[:,"xg": "mean_points"]
        np_team2[- len(df_team2_past):, :-df_team1.loc[:, "weekly_wages_eur":"schedule_round"].shape[1]] = df_team2_past.loc[:,"xg": "mean_points"]


        # if last game of the season, no do not add as no future results to predict
        if (df_team1.iloc[-1].name == df_team1_past.iloc[-1].name) or (df_team2.iloc[-1].name == df_team2_past.iloc[-1].name):
            print(df_team1.iloc[-1].name)
            print(df_team1_past.iloc[-1].name)
            print("hello")
            pass

        # if not last game, add data from last game to input array
        else:
            # 
            np_team1[- len(df_team1_past):, -df_team1.loc[:, "weekly_wages_eur":"schedule_round"].shape[1]:] = df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:,"weekly_wages_eur":"schedule_round"]  
            np_team2[- len(df_team2_past):, -df_team1.loc[:, "weekly_wages_eur":"schedule_round"].shape[1]:] = df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:,"weekly_wages_eur":"schedule_round"]  

            results1 = np.zeros((len(games) * 2, 4))
            results2 = np.zeros((len(games) * 2, 4))
            #print(result1)
            res1 = len(df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:,"result"])
            res2 = len(df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:,"result"])
            results1[-res1:, 0] = df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:,"result"]
            results1[-res1:, 1] = df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:,"gf"]
            results1[-res1:, 2] = df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:,"ga"]
            results1[-res1:, 3] = df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:,"goal_diff"]
            results2[-res2:, 0] = df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:,"result"]
            results2[-res2:, 1] = df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:,"gf"]
            results2[-res2:, 2] = df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:,"ga"]
            results2[-res2:, 3] = df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:,"goal_diff"]
            #fut_feature1 = np.array(df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1,2:].values)
            #fut_feature2 = np.array(df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1,2:].values)
            #print(result1)
            result1 = torch.nn.functional.one_hot(torch.tensor(results1[:,0]).long(), num_classes = 3)
            result2 = torch.nn.functional.one_hot(torch.tensor(results2[:,0]).long(), num_classes = 3)
            lstm_inputs[0].append(torch.tensor(np_team1))
            lstm_inputs[1].append(torch.tensor(np_team2))
            lstm_inputs[2].append(result1)
            lstm_inputs[3].append(result2)
            #lstm_inputs[4].append(fut_feature1)
            #lstm_inputs[5].append(fut_feature2)

    elif (team1 in scale_df[scale_df.season_str == season - 1].team_id.unique()) and (team2 not in scale_df[scale_df.season_str == season - 1].team_id.unique()):
        print("2")
        # get dataframe containing all games of club and reorder according to our need
        #df_team1 = clubs[team1]
        #df_team1 = df_team1[rearrange_list]

        # get season in which game took place
        #season = df_team1[df_team1.match_id == game].season_str.values[0]  

        # create dataframe containing all games of season prior to the game we want to predict
        df_team1_past = df_team1.loc[ \
            df_team1[df_team1.season_str == season - 1].iloc[0].name: # index of first game in seasons
            df_team1.iloc[-2].name,  # index of game we want to predict
            :]  # all columns

        ## Team 2
        # get dataframe containing all games of club and reorder according to our need
        #df_team2 = clubs[team2]
        #df_team2 = df_team2[rearrange_list]

        df_team2_past = df_team2.loc[
            df_team2[df_team2.season_str == season].iloc[0].name:  # index of first game in seasons
            df_team2.iloc[-2].name,  # index of game we want to predict
            :]  # all columns

        # create np array with zero to store data
        np_team1 = np.zeros([len(games)*2, df_team1_past.loc[:,"xg":"schedule_round"].shape[1]])
        np_team2 = np.zeros([len(games)*2, df_team2_past.loc[:,"xg":"schedule_round"].shape[1]])


        # insert data into array (back to front) such that all input into the lstm has the same length (padding)
        np_team1[- len(df_team1_past):, :-df_team1.loc[:, "weekly_wages_eur":"schedule_round"].shape[1]] = df_team1_past.loc[:,"xg": "mean_points"]
        np_team2[- len(df_team2_past):, :-df_team1.loc[:, "weekly_wages_eur":"schedule_round"].shape[1]] = df_team2_past.loc[:,"xg": "mean_points"]


        # if last game of the season, no do not add as no future results to predict
        if (df_team1.iloc[-1].name == df_team1_past.iloc[-1].name) or (df_team2.iloc[-1].name == df_team2_past.iloc[-1].name):
            pass


        # if not last game, add data from last game to input array
        else:
            # 
            np_team1[- len(df_team1_past):, -df_team1.loc[:, "weekly_wages_eur":"schedule_round"].shape[1]:] = df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:,"weekly_wages_eur":"schedule_round"]  
            np_team2[- len(df_team2_past):, -df_team1.loc[:, "weekly_wages_eur":"schedule_round"].shape[1]:] = df_team2.loc[df_team2[df_team2.season_str == season].iloc[1].name:,"weekly_wages_eur":"schedule_round"]  

            results1 = np.zeros((len(games) * 2, 4))
            results2 = np.zeros((len(games) * 2, 4))
            #print(result1)
            res1 = len(df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:,"result"])
            res2 = len(df_team2.loc[df_team2[df_team2.season_str == season].iloc[1].name:,"result"])
            results1[-res1:, 0] = df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:,"result"]
            results1[-res1:, 1] = df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:,"gf"]
            results1[-res1:, 2] = df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:,"ga"]
            results1[-res1:, 3] = df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:,"goal_diff"]
            results2[-res2:, 0] = df_team2.loc[df_team2[df_team2.season_str == season].iloc[1].name:,"result"]
            results2[-res2:, 1] = df_team2.loc[df_team2[df_team2.season_str == season].iloc[1].name:,"gf"]
            results2[-res2:, 2] = df_team2.loc[df_team2[df_team2.season_str == season].iloc[1].name:,"ga"]
            results2[-res2:, 3] = df_team2.loc[df_team2[df_team2.season_str == season].iloc[1].name:,"goal_diff"]
            #fut_feature1 = np.array(df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1,2:].values)
            #fut_feature2 = np.array(df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1,2:].values)
            #print(result1)
            result1 = torch.nn.functional.one_hot(torch.tensor(results1[:,0]).long(), num_classes = 3)
            result2 = torch.nn.functional.one_hot(torch.tensor(results2[:,0]).long(), num_classes = 3)
            lstm_inputs[0].append(torch.tensor(np_team1))
            lstm_inputs[1].append(torch.tensor(np_team2))
            lstm_inputs[2].append(result1)
            lstm_inputs[3].append(result2)
            #lstm_inputs[4].append(fut_feature1)
            #lstm_inputs[5].append(fut_feature2)

    elif (team1 not in scale_df[scale_df.season_str == season - 1].team_id.unique()) and (team2 in scale_df[scale_df.season_str == season - 1].team_id.unique()):
        print("3")
        # get dataframe containing all games of club and reorder according to our need
        #df_team1 = clubs[team1]
        #df_team1 = df_team1[rearrange_list]

        # get season in which game took place
        #season = df_team1[df_team1.match_id == game].season_str.values[0]  

        # create dataframe containing all games of season prior to the game we want to predict
        df_team1_past = df_team1.loc[ \
            df_team1[df_team1.season_str == season].iloc[0].name: # index of first game in seasons
            df_team1.iloc[-2].name,  # index of game we want to predict
            :]  # all columns

        ## Team 2
        # get dataframe containing all games of club and reorder according to our need
        #df_team2 = clubs[team2]
        #df_team2 = df_team2[rearrange_list]

        df_team2_past = df_team2.loc[
            df_team2[df_team2.season_str == season - 1].iloc[0].name:  # index of first game in seasons
            df_team2.iloc[-2].name,  # index of game we want to predict
            :]  # all columns

        # create np array with zero to store data
        np_team1 = np.zeros([len(games)*2, df_team1_past.loc[:,"xg":"schedule_round"].shape[1]])
        print(np_team1)
        np_team2 = np.zeros([len(games)*2, df_team2_past.loc[:,"xg":"schedule_round"].shape[1]])


        # insert data into array (back to front) such that all input into the lstm has the same length (padding)
        np_team1[- len(df_team1_past):, :-df_team1.loc[:, "weekly_wages_eur":"schedule_round"].shape[1]] = df_team1_past.loc[:,"xg": "mean_points"]
        np_team2[- len(df_team2_past):, :-df_team1.loc[:, "weekly_wages_eur":"schedule_round"].shape[1]] = df_team2_past.loc[:,"xg": "mean_points"]


        # if last game of the season, no do not add as no future results to predict
        if (df_team1.iloc[-1].name == df_team1_past.iloc[-1].name) or (df_team2.iloc[-1].name == df_team2_past.iloc[-1].name):
            pass

        # if not last game, add data from last game to input array
        else:
            # 
            np_team1[- len(df_team1_past):, -df_team1.loc[:, "weekly_wages_eur":"schedule_round"].shape[1]:] = df_team1.loc[df_team1[df_team1.season_str == season].iloc[1].name:,"weekly_wages_eur":"schedule_round"]  
            np_team2[- len(df_team2_past):, -df_team1.loc[:, "weekly_wages_eur":"schedule_round"].shape[1]:] = df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:,"weekly_wages_eur":"schedule_round"]  

            results1 = np.zeros((len(games) * 2, 4))
            results2 = np.zeros((len(games) * 2, 4))
            #print(result1)
            res1 = len(df_team1.loc[df_team1[df_team1.season_str == season].iloc[1].name:,"result"])
            res2 = len(df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:,"result"])
            results1[-res1:, 0] = df_team1.loc[df_team1[df_team1.season_str == season].iloc[1].name:,"result"]
            results1[-res1:, 1] = df_team1.loc[df_team1[df_team1.season_str == season].iloc[1].name:,"gf"]
            results1[-res1:, 2] = df_team1.loc[df_team1[df_team1.season_str == season].iloc[1].name:,"ga"]
            results1[-res1:, 3] = df_team1.loc[df_team1[df_team1.season_str == season].iloc[1].name:,"goal_diff"]
            results2[-res2:, 0] = df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:,"result"]
            results2[-res2:, 1] = df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:,"gf"]
            results2[-res2:, 2] = df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:,"ga"]
            results2[-res2:, 3] = df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:,"goal_diff"]
            #fut_feature1 = np.array(df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1,2:].values)
            #fut_feature2 = np.array(df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1,2:].values)
            #print(result1)
            result1 = torch.nn.functional.one_hot(torch.tensor(results1[:,0]).long(), num_classes = 3)
            result2 = torch.nn.functional.one_hot(torch.tensor(results2[:,0]).long(), num_classes = 3)
            lstm_inputs[0].append(torch.tensor(np_team1))
            lstm_inputs[1].append(torch.tensor(np_team2))
            lstm_inputs[2].append(result1)
            lstm_inputs[3].append(result2)
            #lstm_inputs[4].append(fut_feature1)
            #lstm_inputs[5].append(fut_feature2)
    else:
        print("ciao")
        raise KeyError("One or both team/s did not play in last season")
    return lstm_inputs, {team1:[lstm_inputs[0], lstm_inputs[2]]}

In [ ]:
def single_team_input(team1, rearrange_list, scale_df, attendance, venue, ):
    
    lstm_inputs = [[], [], [], [], []] 
    season = int(max(scale_df.season_str))
    
    if team1 not in scale_df[scale_df.season_str == season].team_id.unique():
        #print("Team 1 did not play in the last season")
        raise KeyError("Incorrect input: Team 1 did not play in the last season")
    if team2 not in scale_df[scale_df.season_str == season].team_id.unique():
        #print("Team 2 did not play in the last season")
        raise KeyError("Incorrect input: Team 2 did not play in the last season")
                ## Team 1
                # get ids of participating teams in game 
    team1 = team1
    team2 = team2
    games = scale_df.schedule_round.unique()
    if (team1 in scale_df[scale_df.season_str == season - 1].team_id.unique()) and (team2 in scale_df[scale_df.season_str == season - 1].team_id.unique()):
        print("1")
        # get dataframe containing all games of club and reorder according to our need
        df_team1 = clubs[team1]
        df_team1 = df_team1[rearrange_list]
        # get season in which game took place


        # create dataframe containing all games of season prior to the game we want to predict
        df_team1_past = df_team1.loc[ \
            df_team1[df_team1.season_str == season - 1].iloc[0].name: # index of first game in seasons
            df_team1.iloc[-2].name,  # index of game we want to predict
            :]  # all columns
        
        ## Team 2
        # get dataframe containing all games of club and reorder according to our need
        df_team2 = clubs[team2]
        df_team2 = df_team2[rearrange_list]
        #print("team2", df_team2.shape)
        df_team2_past = df_team2.loc[
            df_team2[df_team2.season_str == season - 1].iloc[0].name:  # index of first game in seasons
            df_team2.iloc[-2].name,  # index of game we want to predict
            :]  # all columns
        
        # create np array with zero to store data
        np_team1 = np.zeros([len(games)*2, df_team1_past.loc[:,"xg":"schedule_round"].shape[1]])
        np_team2 = np.zeros([len(games)*2, df_team2_past.loc[:,"xg":"schedule_round"].shape[1]])
        #print("np_team1",np_team1.shape)

        # insert data into array (back to front) such that all input into the lstm has the same length (padding)
        np_team1[- len(df_team1_past):, :-df_team1.loc[:, "weekly_wages_eur":"schedule_round"].shape[1]] = df_team1_past.loc[:,"xg": "mean_points"]
        np_team2[- len(df_team2_past):, :-df_team1.loc[:, "weekly_wages_eur":"schedule_round"].shape[1]] = df_team2_past.loc[:,"xg": "mean_points"]


        # if last game of the season, no do not add as no future results to predict
        if (df_team1.iloc[-1].name == df_team1_past.iloc[-1].name) or (df_team2.iloc[-1].name == df_team2_past.iloc[-1].name):
            print(df_team1.iloc[-1].name)
            print(df_team1_past.iloc[-1].name)
            print("hello")
            pass

        # if not last game, add data from last game to input array
        else:
            # 
            np_team1[- len(df_team1_past):, -df_team1.loc[:, "weekly_wages_eur":"schedule_round"].shape[1]:] = df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:,"weekly_wages_eur":"schedule_round"]  
            np_team2[- len(df_team2_past):, -df_team1.loc[:, "weekly_wages_eur":"schedule_round"].shape[1]:] = df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:,"weekly_wages_eur":"schedule_round"]  

            results1 = np.zeros((len(games) * 2, 4))
            results2 = np.zeros((len(games) * 2, 4))
            #print(result1)
            res1 = len(df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:,"result"])
            res2 = len(df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:,"result"])
            results1[-res1:, 0] = df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:,"result"]
            results1[-res1:, 1] = df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:,"gf"]
            results1[-res1:, 2] = df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:,"ga"]
            results1[-res1:, 3] = df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:,"goal_diff"]
            results2[-res2:, 0] = df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:,"result"]
            results2[-res2:, 1] = df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:,"gf"]
            results2[-res2:, 2] = df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:,"ga"]
            results2[-res2:, 3] = df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:,"goal_diff"]
            #fut_feature1 = np.array(df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1,2:].values)
            #fut_feature2 = np.array(df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1,2:].values)
            #print(result1)
            result1 = torch.nn.functional.one_hot(torch.tensor(results1[:,0]).long(), num_classes = 3)
            result2 = torch.nn.functional.one_hot(torch.tensor(results2[:,0]).long(), num_classes = 3)
            lstm_inputs[0].append(torch.tensor(np_team1))
            lstm_inputs[1].append(torch.tensor(np_team2))
            lstm_inputs[2].append(result1)
            lstm_inputs[3].append(result2)
            #lstm_inputs[4].append(fut_feature1)
            #lstm_inputs[5].append(fut_feature2)

    elif (team1 in scale_df[scale_df.season_str == season - 1].team_id.unique()) and (team2 not in scale_df[scale_df.season_str == season - 1].team_id.unique()):
        print("2")
        # get dataframe containing all games of club and reorder according to our need
        df_team1 = clubs[team1]
        df_team1 = df_team1[rearrange_list]

        # get season in which game took place
        #season = df_team1[df_team1.match_id == game].season_str.values[0]  

        # create dataframe containing all games of season prior to the game we want to predict
        df_team1_past = df_team1.loc[ \
            df_team1[df_team1.season_str == season - 1].iloc[0].name: # index of first game in seasons
            df_team1.iloc[-2].name,  # index of game we want to predict
            :]  # all columns

        ## Team 2
        # get dataframe containing all games of club and reorder according to our need
        df_team2 = clubs[team2]
        df_team2 = df_team2[rearrange_list]

        df_team2_past = df_team2.loc[
            df_team2[df_team2.season_str == season].iloc[0].name:  # index of first game in seasons
            df_team2.iloc[-2].name,  # index of game we want to predict
            :]  # all columns

        # create np array with zero to store data
        np_team1 = np.zeros([len(games)*2, df_team1_past.loc[:,"xg":"schedule_round"].shape[1]])
        np_team2 = np.zeros([len(games)*2, df_team2_past.loc[:,"xg":"schedule_round"].shape[1]])


        # insert data into array (back to front) such that all input into the lstm has the same length (padding)
        np_team1[- len(df_team1_past):, :-df_team1.loc[:, "weekly_wages_eur":"schedule_round"].shape[1]] = df_team1_past.loc[:,"xg": "mean_points"]
        np_team2[- len(df_team2_past):, :-df_team1.loc[:, "weekly_wages_eur":"schedule_round"].shape[1]] = df_team2_past.loc[:,"xg": "mean_points"]


        # if last game of the season, no do not add as no future results to predict
        if (df_team1.iloc[-1].name == df_team1_past.iloc[-1].name) or (df_team2.iloc[-1].name == df_team2_past.iloc[-1].name):
            pass


        # if not last game, add data from last game to input array
        else:
            # 
            np_team1[- len(df_team1_past):, -df_team1.loc[:, "weekly_wages_eur":"schedule_round"].shape[1]:] = df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:,"weekly_wages_eur":"schedule_round"]  
            np_team2[- len(df_team2_past):, -df_team1.loc[:, "weekly_wages_eur":"schedule_round"].shape[1]:] = df_team2.loc[df_team2[df_team2.season_str == season].iloc[1].name:,"weekly_wages_eur":"schedule_round"]  

            results1 = np.zeros((len(games) * 2, 4))
            results2 = np.zeros((len(games) * 2, 4))
            #print(result1)
            res1 = len(df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:,"result"])
            res2 = len(df_team2.loc[df_team2[df_team2.season_str == season].iloc[1].name:,"result"])
            results1[-res1:, 0] = df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:,"result"]
            results1[-res1:, 1] = df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:,"gf"]
            results1[-res1:, 2] = df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:,"ga"]
            results1[-res1:, 3] = df_team1.loc[df_team1[df_team1.season_str == season - 1].iloc[1].name:,"goal_diff"]
            results2[-res2:, 0] = df_team2.loc[df_team2[df_team2.season_str == season].iloc[1].name:,"result"]
            results2[-res2:, 1] = df_team2.loc[df_team2[df_team2.season_str == season].iloc[1].name:,"gf"]
            results2[-res2:, 2] = df_team2.loc[df_team2[df_team2.season_str == season].iloc[1].name:,"ga"]
            results2[-res2:, 3] = df_team2.loc[df_team2[df_team2.season_str == season].iloc[1].name:,"goal_diff"]
            #fut_feature1 = np.array(df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1,2:].values)
            #fut_feature2 = np.array(df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1,2:].values)
            #print(result1)
            result1 = torch.nn.functional.one_hot(torch.tensor(results1[:,0]).long(), num_classes = 3)
            result2 = torch.nn.functional.one_hot(torch.tensor(results2[:,0]).long(), num_classes = 3)
            lstm_inputs[0].append(torch.tensor(np_team1))
            lstm_inputs[1].append(torch.tensor(np_team2))
            lstm_inputs[2].append(result1)
            lstm_inputs[3].append(result2)
            #lstm_inputs[4].append(fut_feature1)
            #lstm_inputs[5].append(fut_feature2)

    elif (team1 not in scale_df[scale_df.season_str == season - 1].team_id.unique()) and (team2 in scale_df[scale_df.season_str == season - 1].team_id.unique()):
        print("3")
        # get dataframe containing all games of club and reorder according to our need
        df_team1 = clubs[team1]
        df_team1 = df_team1[rearrange_list]

        # get season in which game took place
        #season = df_team1[df_team1.match_id == game].season_str.values[0]  

        # create dataframe containing all games of season prior to the game we want to predict
        df_team1_past = df_team1.loc[ \
            df_team1[df_team1.season_str == season].iloc[0].name: # index of first game in seasons
            df_team1.iloc[-2].name,  # index of game we want to predict
            :]  # all columns

        ## Team 2
        # get dataframe containing all games of club and reorder according to our need
        df_team2 = clubs[team2]
        df_team2 = df_team2[rearrange_list]

        df_team2_past = df_team2.loc[
            df_team2[df_team2.season_str == season - 1].iloc[0].name:  # index of first game in seasons
            df_team2.iloc[-2].name,  # index of game we want to predict
            :]  # all columns

        # create np array with zero to store data
        np_team1 = np.zeros([len(games)*2, df_team1_past.loc[:,"xg":"schedule_round"].shape[1]])
        print(np_team1)
        np_team2 = np.zeros([len(games)*2, df_team2_past.loc[:,"xg":"schedule_round"].shape[1]])


        # insert data into array (back to front) such that all input into the lstm has the same length (padding)
        np_team1[- len(df_team1_past):, :-df_team1.loc[:, "weekly_wages_eur":"schedule_round"].shape[1]] = df_team1_past.loc[:,"xg": "mean_points"]
        np_team2[- len(df_team2_past):, :-df_team1.loc[:, "weekly_wages_eur":"schedule_round"].shape[1]] = df_team2_past.loc[:,"xg": "mean_points"]


        # if last game of the season, no do not add as no future results to predict
        if (df_team1.iloc[-1].name == df_team1_past.iloc[-1].name) or (df_team2.iloc[-1].name == df_team2_past.iloc[-1].name):
            pass

        # if not last game, add data from last game to input array
        else:
            # 
            np_team1[- len(df_team1_past):, -df_team1.loc[:, "weekly_wages_eur":"schedule_round"].shape[1]:] = df_team1.loc[df_team1[df_team1.season_str == season].iloc[1].name:,"weekly_wages_eur":"schedule_round"]  
            np_team2[- len(df_team2_past):, -df_team1.loc[:, "weekly_wages_eur":"schedule_round"].shape[1]:] = df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:,"weekly_wages_eur":"schedule_round"]  

            results1 = np.zeros((len(games) * 2, 4))
            results2 = np.zeros((len(games) * 2, 4))
            #print(result1)
            res1 = len(df_team1.loc[df_team1[df_team1.season_str == season].iloc[1].name:,"result"])
            res2 = len(df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:,"result"])
            results1[-res1:, 0] = df_team1.loc[df_team1[df_team1.season_str == season].iloc[1].name:,"result"]
            results1[-res1:, 1] = df_team1.loc[df_team1[df_team1.season_str == season].iloc[1].name:,"gf"]
            results1[-res1:, 2] = df_team1.loc[df_team1[df_team1.season_str == season].iloc[1].name:,"ga"]
            results1[-res1:, 3] = df_team1.loc[df_team1[df_team1.season_str == season].iloc[1].name:,"goal_diff"]
            results2[-res2:, 0] = df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:,"result"]
            results2[-res2:, 1] = df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:,"gf"]
            results2[-res2:, 2] = df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:,"ga"]
            results2[-res2:, 3] = df_team2.loc[df_team2[df_team2.season_str == season - 1].iloc[1].name:,"goal_diff"]
            #fut_feature1 = np.array(df_team1.loc[df_team1[df_team1.match_id == game].index.values[0]:,:].iloc[1,2:].values)
            #fut_feature2 = np.array(df_team2.loc[df_team2[df_team2.match_id == game].index.values[0]:,:].iloc[1,2:].values)
            #print(result1)
            result1 = torch.nn.functional.one_hot(torch.tensor(results1[:,0]).long(), num_classes = 3)
            result2 = torch.nn.functional.one_hot(torch.tensor(results2[:,0]).long(), num_classes = 3)
            lstm_inputs[0].append(torch.tensor(np_team1))
            lstm_inputs[1].append(torch.tensor(np_team2))
            lstm_inputs[2].append(result1)
            lstm_inputs[3].append(result2)
            #lstm_inputs[4].append(fut_feature1)
            #lstm_inputs[5].append(fut_feature2)
    else:
        print("ciao")
        raise KeyError("One or both team/s did not play in last season")
    return lstm_inputs, {team1:[lstm_inputs[0], lstm_inputs[2]]}

In [ ]:
two_team_inputs(42, 69, rearange_list, scale_df)

In [ ]:
train_inputs = inputs_2seas(game_train, clubs, rearange_list, scale_df)
valid_inputs = inputs_2seas(game_valid, clubs, rearange_list, scale_df)

In [ ]:
train_for_loader = data_to_lstm(train_inputs)
valid_for_loader = data_to_lstm(valid_inputs)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_for_loader, batch_size = 32, drop_last = True)
test_loader = torch.utils.data.DataLoader(valid_for_loader, batch_size = 32, drop_last = True)

In [ ]:
len_input = train_inputs[0][0].shape[1]
lr = 2e-5
#wdc = [0.2, 0.15, 0.1, 0.05, 0]
#lrs = [1e-3, 1e-4, 1e-5, 1e-6]
#for lr in lrs:
#for wd in wdc:
net = Sport_pred_2LSTM_1(len_input, len_input, 3, 2)
#print(f"\n\nLR: {lr}, WD: {wd}")
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr = lr)#, weight_decay = 0.01)

net.eval()
accur = 0
for step, ((input1, input2), (result1, result2)) in enumerate(test_loader):
    pred = net(input1)
    #print(pred.shape)
    #print(result1.shape)
    pred = torch.argmax(pred[:,-1,:], dim = 1)
    result = result1[:,-1,:]
    result = torch.argmax(result, dim = 1)
    accur += pred.eq(result).sum().item()

    pred = net(input2)
    pred = torch.argmax(pred[:,-1,:], dim = 1)
    result2 = result2[:,-1,:]
    result2 = torch.argmax(result2, dim = 1)
    accur += pred.eq(result2).sum().item()
print(f"Loss accuracy training: {100 * accur /((step + 1) * 64)}%")

for epoch in range(5):
    losses_val = []
    accuracies = []

    net.train()
    print(f"Epoch: {epoch}")
    for step, ((input1, input2), (result1, result2)) in enumerate(train_loader):
        net.zero_grad()
        pred = net(input1)
        #result = torch.nn.functional.one_hot(result.to(torch.int64), num_classes = 3).to(torch.float32)
        #print(f"pred {pred.dtype}, result {result.dtype}")
        #result1 = result1[:,-1,:]
        loss = criterion(pred.float(), result1.float())
        loss.backward()
        optimizer.step()

        net.zero_grad()
        pred = net(input2)
        #result2 = result2[:,-1,:]
        #result = torch.nn.functional.one_hot(result.to(torch.int64), num_classes = 3).to(torch.float32)
        #print(f"pred {pred.dtype}, result {result.dtype}")
        loss = criterion(pred.float(), result2.float())
        loss.backward()
        optimizer.step()

    net.eval()
    loss = 0
    accur = 0
    for step, ((input1, input2), (result1, result2)) in enumerate(test_loader):
        pred = net(input1)
        #print(pred)
        #print(pred.shape)
        result1 = result1[:,-1,:]
        pred = pred[:,-1,:]
        loss += criterion(pred, result1).item()
        #
        #print(pred)
        #print(pred.shape)
        pred = torch.argmax(pred, dim = 1)
        #print(pred)
        #print(pred.shape)

        result1 = torch.argmax(result1, dim = 1)
        accur += pred.eq(result1).sum().item()
       # if epoch == 9:
        #    print("pred", pred)
         #   print("result", result1)
        pred = net(input2)
        result2 = result2[:,-1,:]
        pred = pred[:,-1,:]
        loss += criterion(pred, result2).item()
        #pred = pred[:,-1,:]
        pred = torch.argmax(pred, dim = 1)
        result2  = torch.argmax(result2, dim = 1)
        accur += pred.eq(result2).sum().item()
    losses_val.append(loss)
    accuracy = 100 * (accur /((step + 1) * 64))
    os.makedirs(os.path.dirname(f"./models/sequence_model_2seas/LSTM/{lr}/accur_{round(accuracy, 2)}"), exist_ok = True)
    torch.save(net.state_dict(), f"./models/sequence_model_2seas/LSTM/{lr}/accur_{round(accuracy, 2)}")
    accuracies.append(accuracy)
    print(f"Loss {loss} accuracy {accuracy}%") 
print("done")


In [ ]:
len_input = train_inputs[0][0].shape[1]
lr = 2e-5
#wdc = [0.2, 0.15, 0.1, 0.05, 0]
#lrs = [1e-3, 1e-4, 1e-5, 1e-6]
#for lr in lrs:
#for wd in wdc:
net = Sport_pred_2LSTM_1(len_input, len_input, 3, 2)
#print(f"\n\nLR: {lr}, WD: {wd}")
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr = lr)#, weight_decay = 0.01)

net.eval()
accur = 0
for step, ((input1, input2), (result1, result2)) in enumerate(test_loader):
    pred1 = net(input1)[:,-1,:]
    pred2 = net(input2)[:,-1,:]
    result = result1[:,-1,:]
    result = torch.argmax(result, dim = 1)
    predicted = pred_add(pred1, pred2) #predict(pred1, pred2, result_dict)
    predicted = torch.argmax(predicted, dim = 1)
    accur += predicted.eq(result).sum().item()
print(f"Loss accuracy training: {100 * accur /((step + 1) * 32)}%")

for epoch in range(6):
    losses_val = []
    accuracies = []

    net.train()
    print(f"Epoch: {epoch}")
    for step, ((input1, input2), (result1, result2)) in enumerate(train_loader):
        net.zero_grad()
        pred = net(input1)
        loss = criterion(pred.float(), result1.float())
        loss.backward()
        optimizer.step()

        net.zero_grad()
        pred = net(input2)
        loss = criterion(pred.float(), result2.float())
        loss.backward()
        optimizer.step()

    net.eval()
    loss = 0
    accur = 0
    for step, ((input1, input2), (result1, result2)) in enumerate(test_loader):
        pred1 = net(input1)[:,-1,:]
        pred2 = net(input2)[:,-1,:]
        result = result1[:,-1,:]
        result  = torch.argmax(result, dim = 1)
        predicted = pred_add(pred1, pred2) #predict(pred1, pred2, result_dict)
        predicted = torch.argmax(predicted, dim = 1)
        accur += predicted.eq(result).sum().item()
    losses_val.append(loss)
    accuracy = 100 * (accur /((step + 1) * 32))
    os.makedirs(os.path.dirname(f"./models/sequence_model_2seas/LSTM/accur_{round(accuracy, 2)}"), exist_ok = True)
    torch.save(net.state_dict(), f"./models/sequence_model_2seas/LSTM/accur_{round(accuracy, 2)}")
    accuracies.append(accuracy)
    print(f"Loss {loss} accuracy {accuracy}%") 
print("done")


In [ ]:
len_input = train_inputs[0][0].shape[1]
#wdc = [0.2, 0.15, 0.1, 0.05, 0]
lrs = [1e-3, 1e-4, 1e-5, 1e-6]
for lr in lrs:
    print(lr)
    #for wd in wdc:
    net = Sport_pred_2LSTM_1(len_input, len_input, 3, 2)
    #print(f"\n\nLR: {lr}, WD: {wd}")
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(net.parameters(), lr = lr)#, weight_decay = 0.01)

    net.eval()
    accur = 0
    for step, ((input1, input2), (result1, result2)) in enumerate(test_loader):
        pred = net(input1)
        #print(pred.shape)
        #print(result1.shape)
        pred1 = torch.argmax(pred[:,-1,:], dim = 1)
        result = result1[:,-1,:]
        result = torch.argmax(result, dim = 1)
        #print(result.shape)
        #print(pred.shape)
        accur += pred1.eq(result).sum().item()

        pred2 = net(input2)
        pred = torch.argmax(pred[:,-1,:], dim = 1)
        result2 = result2[:,-1,:]
        result2 = torch.argmax(result2, dim = 1)
        #predicted = predict(pred1, pred2)
        accur += pred.eq(result2).sum().item()
    print(f"Loss accuracy training: {100 * accur /((step + 1) * 64)}%")

    for epoch in range(6):
        losses_val = []
        accuracies = []

        net.train()
        print(f"Epoch: {epoch}")
        for step, ((input1, input2), (result1, result2)) in enumerate(train_loader):
            net.zero_grad()
            pred = net(input1)
            #result = torch.nn.functional.one_hot(result.to(torch.int64), num_classes = 3).to(torch.float32)
            #print(f"pred {pred.dtype}, result {result.dtype}")
            #result1 = result1[:,-1,:]
            loss = criterion(pred.float(), result1.float())
            loss.backward()
            optimizer.step()

            net.zero_grad()
            pred = net(input2)
            #result2 = result2[:,-1,:]
            #result = torch.nn.functional.one_hot(result.to(torch.int64), num_classes = 3).to(torch.float32)
            #print(f"pred {pred.dtype}, result {result.dtype}")
            loss = criterion(pred.float(), result2.float())
            loss.backward()
            optimizer.step()

        net.eval()
        loss = 0
        accur = 0
        for step, ((input1, input2), (result1, result2)) in enumerate(test_loader):
            pred1 = net(input1)
            #print(pred)
            #print(pred.shape)
            result1 = result1[:,-1,:]
            pred = pred1[:,-1,:]
            loss += criterion(pred, result1).item()
            #print(result1.shape)
            #print(pred)
            #print(pred.shape)
            pred = torch.argmax(pred, dim = 1)
            #print(pred)
            #print(pred.shape)

            result1 = torch.argmax(result1, dim = 1)
            #result1
            accur += pred.eq(result1).sum().item()
           # if epoch == 9:
            #    print("pred", pred)
             #   print("result", result1)
            pred2 = net(input2)
            result2 = result2[:,-1,:]
            pred = pred2[:,-1,:]
            loss += criterion(pred, result2).item()
            #pred = pred[:,-1,:]
            pred = torch.argmax(pred, dim = 1)
            #result  = torch.argmax(result1, dim = 1)
            #predicted = torch.nn.functional.one_hot(predict(pred1, pred2), num_classes = 3)
            #print(predicted.shape)
            #print(result.shape)
            result2 = torch.argmax(result2, dim = 1)
            accur += pred.eq(result2).sum().item()
        losses_val.append(loss)
        accuracy = 100 * (accur /((step + 1) * 64))
        os.makedirs(os.path.dirname(f"./models/sequence_model_2seas/LSTM/{lr}/accur_{round(accuracy, 2)}"), exist_ok = True)
        torch.save(net.state_dict(), f"./models/sequence_model_2seas/LSTM/{lr}/accur_{round(accuracy, 2)}")
        accuracies.append(accuracy)
        print(f"Loss {loss} accuracy {accuracy}%") 
    print("done")

In [ ]:
def pred_add(pred1, pred2):
    pred2 = torch.index_select(pred2, 1, torch.LongTensor([1,0,2]))
    pred = pred1 + pred2
    result = torch.nn.functional.softmax(pred)
    return result

In [ ]:
len_input = train_inputs[0][0].shape[1]
#wdc = [0.2, 0.15, 0.1, 0.05, 0]
#lrs = [1e-3, 1e-4, 1e-5, 1e-6]
#for lr in lrs:
#for wd in wdc:
net = Sport_pred_2LSTM_1(len_input, len_input, 3, 2)
#print(f"\n\nLR: {lr}, WD: {wd}")
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr = 2e-5)#, weight_decay = 0.01)

net.eval()
accur = 0
for step, ((input1, input2), (result1, result2)) in enumerate(test_loader):
    pred = net(input1)
    #print(pred.shape)
    #print(result1.shape)
    pred1 = torch.argmax(pred[:,-1,:], dim = 1)
    result = result1[:,-1,:]
    result = torch.argmax(result, dim = 1)
    #print(result.shape)
    #print(pred.shape)
    accur += pred1.eq(result).sum().item()

    pred2 = net(input2)
    pred = torch.argmax(pred[:,-1,:], dim = 1)
    result2 = result2[:,-1,:]
    result2 = torch.argmax(result2, dim = 1)
    #predicted = predict(pred1, pred2)
    accur += pred.eq(result2).sum().item()
print(f"Loss accuracy training: {100 * accur /((step + 1) * 64)}%")

for epoch in range(6):
    losses_val = []
    accuracies = []

    net.train()
    print(f"Epoch: {epoch}")
    for step, ((input1, input2), (result1, result2)) in enumerate(train_loader):
        net.zero_grad()
        pred = net(input1)
        #result = torch.nn.functional.one_hot(result.to(torch.int64), num_classes = 3).to(torch.float32)
        #print(f"pred {pred.dtype}, result {result.dtype}")
        #result1 = result1[:,-1,:]
        loss = criterion(pred.float(), result1.float())
        loss.backward()
        optimizer.step()

        net.zero_grad()
        pred = net(input2)
        #result2 = result2[:,-1,:]
        #result = torch.nn.functional.one_hot(result.to(torch.int64), num_classes = 3).to(torch.float32)
        #print(f"pred {pred.dtype}, result {result.dtype}")
        loss = criterion(pred.float(), result2.float())
        loss.backward()
        optimizer.step()

    net.eval()
    loss = 0
    accur = 0
    for step, ((input1, input2), (result1, result2)) in enumerate(test_loader):
        pred1 = net(input1)
        #print(pred)
        #print(pred.shape)
        result1 = result1[:,-1,:]
        pred = pred1[:,-1,:]
        loss += criterion(pred, result1).item()
        #print(result1.shape)
        #print(pred)
        #print(pred.shape)
        pred = torch.argmax(pred, dim = 1)
        #print(pred)
        #print(pred.shape)

        result1 = torch.argmax(result1, dim = 1)
        #result1
        accur += pred.eq(result1).sum().item()
       # if epoch == 9:
        #    print("pred", pred)
         #   print("result", result1)
        pred2 = net(input2)
        result2 = result2[:,-1,:]
        pred = pred2[:,-1,:]
        loss += criterion(pred, result2).item()
        #pred = pred[:,-1,:]
        pred = torch.argmax(pred, dim = 1)
        #result  = torch.argmax(result1, dim = 1)
        #predicted = torch.nn.functional.one_hot(predict(pred1, pred2), num_classes = 3)
        #print(predicted.shape)
        #print(result.shape)
        accur += pred.eq(result).sum().item()
    losses_val.append(loss)
    accuracy = 100 * (accur /((step + 1) * 64))
    os.makedirs(os.path.dirname(f"./models/sequence_model_2seas/LSTM/accur_{round(accuracy, 2)}"), exist_ok = True)
    torch.save(net.state_dict(), f"./models/sequence_model_2seas/LSTM/accur_{round(accuracy, 2)}")
    accuracies.append(accuracy)
    print(f"Loss {loss} accuracy {accuracy}%") 
print("done")

In [ ]:
class Sport_pred_2GRU_1(torch.nn.Module):
    def __init__(self,n_features, hidden, num_classes, num_layers = 1):
        super(Sport_pred_2GRU_1, self).__init__()
        self.n_features = n_features 
        self.num_classes = num_classes # number of classes (win, draw, lose)
        self.n_hidden = hidden # number of hidden states
        self.n_layers = num_layers # number of LSTM layers (stacked)
        
        # two separate lstms to account for every teams history
        self.l_lstm1 = torch.nn.GRU(input_size = n_features, 
                             hidden_size = self.n_hidden,
                             num_layers = self.n_layers,
                             batch_first = True)
        

        # classic neural net to process outcomes
        self.l_linear1 = torch.nn.Linear(self.n_hidden, self.num_classes)
        #self.l_linear2 = torch.nn.Linear(self.n_hidden, self.num_classes)
        
        
        self.soft = torch.nn.Softmax()
        self.relu = torch.nn.ReLU()
        self.sigm = torch.nn.Sigmoid()


    def forward(self, x):
        # convert input to fit the model
        x = x.to(torch.float32)
        x = torch.nan_to_num(x, nan = 0.0)
        
        
        # initialize values for two lstms
        #h01 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(x.dtype)
        #c01 = torch.zeros(self.n_layers*1, 32, self.n_hidden).to(x.dtype)
        
        
        # run data through lstm and yield output
        lstm_out1,(ht1, ct1) = self.l_lstm1(x)#,(h01, c01))
        ht1 = ht1.squeeze()#[-1,:]
        ct1 = ct1.squeeze()#[-1,:]
        #print(lstm_out1.shape)
        out = lstm_out1#[:,-1,:]#.contiguous().view(-1, self.n_hidden)
        #print(out.shape)
        
        # run lstm output through nn to predict outcome
        result = self.l_linear1(out)
        #result = self.l_linear2(result)
        #result = self.soft(result)
        #result = self.sigm(result)
        return result